# Initialize Everything

In [ ]:
from pymongo import MongoClient
import pandas as pd

## Read MongoDB

In [ ]:
client = MongoClient()
db = client["test"]
collection = db["matchapps"]

## Flatten, reorganize, and clean Data

In [ ]:
df = pd.json_normalize(collection.find())

column_order = [
    "_id",
    "metadata.matchNumber",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "trapNotes",
    "autoNotes.near",
    "autoNotes.mid",
    "autoNotes.far",
    "autoNotes.amp",
    "autoNotes.miss",
    "teleNotes.near",
    "teleNotes.mid",
    "teleNotes.far",
    "teleNotes.amp",
    "teleNotes.miss"   
]

df = df[column_order]
df

## Initialize Output DF

In [ ]:
output_df = pd.DataFrame()

# Function and Variable Initializations

## Check Alliance Harmonization Function Definition

In [ ]:
def check_team_harmonization_in_alliance(t1, t2, t3):
    t1_status, t2_status, t3_status = False, False, False
    if t1 == t2:
        t1_status = True
        t2_status = True
    if t1 == t3:
        t1_status = True
        t3_status = True
    if t2 == t3:
        t2_status = True
        t3_status = True
    return t1_status, t2_status, t3_status

## Complete List of all Teams in DF

In [ ]:
team_list = df["metadata.robotTeam"].unique().tolist()

## Complete Range of All Matches

In [ ]:
match_range = df["metadata.matchNumber"].max()
match_range

# Analyze Data

## Auto Data Analysis Calculations

### Auto Speaker Notes Average (near, mid, far)

In [ ]:
output_df["Auto Speaker Notes Near Avg"] = df.groupby("metadata.robotTeam")["autoNotes.near"].mean()
output_df["Auto Speaker Notes Mid Avg"] = df.groupby("metadata.robotTeam")["autoNotes.mid"].mean()
output_df["Auto Speaker Notes Far Avg"] = df.groupby("metadata.robotTeam")["autoNotes.far"].mean()

### Auto Amp Notes Average

In [ ]:
output_df["Auto Notes Amp Avg"] = df.groupby("metadata.robotTeam")["autoNotes.amp"].mean()

### Auto Percent of Miss Notes

In [ ]:
output_df["Auto Percent of Missed Notes"] = None
for team in team_list:
    missed_notes = df[df["metadata.robotTeam"] == team]["autoNotes.miss"].sum()
    total_notes = df[df["metadata.robotTeam"] == team]["autoNotes.near"].sum()
    + df[df["metadata.robotTeam"] == team]["autoNotes.mid"].sum()
    + df[df["metadata.robotTeam"] == team]["autoNotes.far"].sum()
    + df[df["metadata.robotTeam"] == team]["autoNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes

## Auto Data Analysis Calculations

### Tele Speaker Notes Average (near, mid, far)

In [ ]:
output_df["Tele Speaker Notes Near Avg"] = df.groupby("metadata.robotTeam")["teleNotes.near"].mean()
output_df["Tele Speaker Notes Mid Avg"] = df.groupby("metadata.robotTeam")["teleNotes.mid"].mean()
output_df["Tele Speaker Notes Far Avg"] = df.groupby("metadata.robotTeam")["teleNotes.far"].mean()

### Tele Amp Notes Average

In [ ]:
output_df["Tele Notes Amp Avg"] = df.groupby("metadata.robotTeam")["teleNotes.amp"].mean()

### Tele Percent of Missed Notes

In [ ]:
output_df["Tele Percent of Missed Notes"] = None
for team in team_list:
    missed_notes = df[df["metadata.robotTeam"] == team]["teleNotes.miss"].sum()
    total_notes = df[df["metadata.robotTeam"] == team]["teleNotes.near"].sum()
    + df[df["metadata.robotTeam"] == team]["teleNotes.mid"].sum()
    + df[df["metadata.robotTeam"] == team]["teleNotes.far"].sum()
    + df[df["metadata.robotTeam"] == team]["teleNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes

## Other Data Analysis Calculations

### Trap Notes

In [ ]:
output_df["Trap Notes Avg"] = df.groupby("metadata.robotTeam")["trapNotes"].mean()

### Percent of Successful Climbs and Successful Climbs Total

In [ ]:
output_df["Percent of Successful Climbs"] = None
output_df["Successful Climbs Total"] = None

for team in team_list:
    climb_list = df.loc[df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        output_df.at[team, "Percent of Successful Climbs"] = 0
        
    else:
        output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs
    
    output_df.at[team, "Successful Climbs Total"] = successful_climbs

### Harmonized Climbs Total

In [ ]:
output_df["Harmonized Climbs Total"] = 0

for match in range(match_range):
    
    match_number_df = df['metadata.matchNumber'] == match
    robots = df[match_number_df]
    robots.sort_values("metadata.robotPosition")
    
    teams_harmonized = []
    
    for i in range(6):
        teams_harmonized.append([robots.at[i + 1, "Climb"], False])

    teams_harmonized[0][1], teams_harmonized[1][1], teams_harmonized[2][1] = check_team_harmonization_in_alliance(teams_harmonized[0][0], teams_harmonized[1][0], teams_harmonized[2][0])
    teams_harmonized[3][1], teams_harmonized[4][1], teams_harmonized[5][1] = check_team_harmonization_in_alliance(teams_harmonized[3][0], teams_harmonized[4][0], teams_harmonized[5][0])

    for checked_team in teams_harmonized:
        if checked_team[1]:
            output_df.at[checked_team[0], "Harmonized Climbs Total"] += 1

### Percent of Harmonized Climbs out of Successful Climbs

In [ ]:
output_df["Percent of Harmonized Climbs out of Successful Climbs"] = 0

for team in team_list:
    output_df.at[team, "Percent of Harmonized Climbs out of Successful Climbs"] = output_df.at[team, "Harmonized Climbs Total"] / output_df.at[team, "Successful Climbs Total"]

### Percent of Times Robot Left Starting Zone During Autos

In [ ]:
output_df["Percent of Times Robot Left Starting Zone During Autos"] = df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

# Finalize and Send Data

## Send Data

In [ ]:
output_df
#df.to_json("../server/static/analyzed_data.json", orient='records', default_handler=str)